In [1]:
!pip install unsloth
!pip install transformers
!pip install trl
!pip install rouge
!pip install scikit-learn

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.8/57.8 kB 3.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.0/44.0 kB 3.0 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of torchvision to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 188.8/188.8 kB 11.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.0/10.0 MB 76.6 MB/s eta 0:00:00:00:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 253.1/253.1 MB 6.9 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.9/318.9 kB 18.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 107.1/107.1 kB 8.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.3/43.3 MB 40.4 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 766.7/766.7 MB 2.0 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 2.7 MB/s eta 0:00:000:00:010

In [2]:
import torch
from unsloth import FastLanguageModel
from datasets import load_dataset
from transformers import AutoTokenizer
from rouge import Rouge
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np
from tqdm import tqdm
import json


# Initialize the same prompt template
finqa_prompt = """Below contains texts before table (pre-text), text after the table (post-text) and the table itself with a question that you must answer by providing only the mathematical or logical operations needed to solve it along with the result.
### Pre-text:
{}
### Table:
{}
### Post-text:
{}
### Question:
{}
### Response:
{}"""

def format_test_input(example):
    """Format a test example for inference"""
    pre_text = ' '.join(example['pre_text']) if example['pre_text'] else ""
    post_text = ' '.join(example['post_text']) if example['post_text'] else ""
    table = "\n".join(" | ".join(str(cell) for cell in row) for row in example['table']) if example['table'] else ""
    
    return finqa_prompt.format(
        pre_text,
        table,
        post_text,
        example['question'],
        ""  # Empty response for generation
    )

def extract_operations_result(response):
    """Extract operations and result from model response"""
    if "### Response:" in response:
        response = response.split("### Response:")[-1].strip()
    
    if "=" in response:
        parts = response.split("=", 1)
        operations = parts[0].strip()
        result = parts[1].strip() if len(parts) > 1 else None
        return operations, result
    
    return None, None

def evaluate_sample(model, tokenizer, example):
    """Evaluate model on a single example"""
    input_text = format_test_input(example)
    inputs = tokenizer([input_text], return_tensors="pt").to("cuda")
    
    try:
        outputs = model.generate(
            **inputs,
            max_new_tokens=128,
            temperature=1e-10,
            # top_p=0.9,
            use_cache = True,
            pad_token_id=tokenizer.eos_token_id,
            eos_token_id=tokenizer.eos_token_id,
            # repetition_penalty=1.2
        )
        
        response = tokenizer.decode(outputs[0], skip_special_tokens=True)
        
        generated_ops, generated_result = extract_operations_result(response)
        expected_ops = example['program_re']
        expected_result = str(example['answer'])
        
        # Calculate ROUGE scores
        rouge = Rouge()
        try:
            rouge_scores = rouge.get_scores(
                generated_ops or "",
                expected_ops
            )[0]
        except:
            rouge_scores = {
                'rouge-1': {'f': 0.0},
                'rouge-2': {'f': 0.0},
                'rouge-l': {'f': 0.0}
            }
        
        # Calculate Cosine similarity
        vectorizer = TfidfVectorizer()
        try:
            tfidf_matrix = vectorizer.fit_transform([
                generated_ops or "",
                expected_ops
            ])
            cosine_sim = cosine_similarity(tfidf_matrix[0:1], tfidf_matrix[1:2])[0][0]
        except:
            cosine_sim = 0.0
        
        # Check if the generated result matches the expected result
        result_match = False
        if generated_result and expected_result:
            try:
                # Try to convert both to float for numerical comparison
                gen_float = float(generated_result.replace(',', ''))
                exp_float = float(expected_result.replace(',', ''))
                result_match = abs(gen_float - exp_float) < 1e-5  # Allow small floating-point differences
            except:
                # If conversion fails, do string comparison
                result_match = generated_result.strip() == expected_result.strip()
        
        return {
            'question': example['question'],
            'expected_equation': f"{expected_ops} = {expected_result}",
            'generated_equation': f"{generated_ops} = {generated_result}" if generated_ops else None,
            'rouge1_f': rouge_scores['rouge-1']['f'],
            'rouge2_f': rouge_scores['rouge-2']['f'],
            'rougeL_f': rouge_scores['rouge-l']['f'],
            'cosine_similarity': cosine_sim,
            'result_match': result_match,
            'raw_response': response
        }
    
    except Exception as e:
        print(f"Error processing example: {str(e)}")
        return None

def evaluate_full_dataset():
    """Evaluate model on the entire test set"""
    print("Loading model and tokenizer from HuggingFace...")
    model_name = "n3Er/qwen2.5-7b-instruct-finqa-ht"
    
    # Load tokenizer
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    
    # Load model with unsloth optimizations
    model, tokenizer = FastLanguageModel.from_pretrained(
        model_name=model_name,
        max_seq_length=2048,
        dtype=None,
        load_in_4bit=True,
    )
    
    # Set model to inference mode
    FastLanguageModel.for_inference(model)
    
    print("Loading test dataset...")
    dataset = load_dataset("ibm/finqa", split="test", trust_remote_code=True)
    
    # Filter valid examples
    valid_examples = [
        ex for ex in dataset 
        if ex['program_re'] and ex['answer'] is not None
    ]
    
    results = []
    total_samples = len(valid_examples)
    print(f"\nEvaluating {total_samples} samples...")
    
    # Use tqdm for progress bar
    for sample in tqdm(valid_examples, desc="Evaluating"):
        result = evaluate_sample(model, tokenizer, sample)
        if result:
            results.append(result)
    
    # Calculate metrics
    avg_metrics = {
        'rouge1_f': np.mean([r['rouge1_f'] for r in results]),
        'rouge2_f': np.mean([r['rouge2_f'] for r in results]),
        'rougeL_f': np.mean([r['rougeL_f'] for r in results]),
        'cosine_similarity': np.mean([r['cosine_similarity'] for r in results]),
        'result_accuracy': np.mean([1 if r['result_match'] else 0 for r in results]),
        'total_samples': len(results),
        'successful_evaluations': len(results),
        'failed_evaluations': total_samples - len(results)
    }
    
    # Print results
    print("\nOverall Results:")
    print("=" * 50)
    print(f"Total samples processed: {avg_metrics['total_samples']}")
    print(f"Successful evaluations: {avg_metrics['successful_evaluations']}")
    print(f"Failed evaluations: {avg_metrics['failed_evaluations']}")
    print(f"Average ROUGE-1 F1: {avg_metrics['rouge1_f']:.3f}")
    print(f"Average ROUGE-2 F1: {avg_metrics['rouge2_f']:.3f}")
    print(f"Average ROUGE-L F1: {avg_metrics['rougeL_f']:.3f}")
    print(f"Average Cosine Similarity: {avg_metrics['cosine_similarity']:.3f}")
    print(f"Result Accuracy: {avg_metrics['result_accuracy']:.3f}")
    
    # Save results
    output_file = "evaluation_results.json"
    
    with open(output_file, 'w') as f:
        json.dump({
            'metrics': avg_metrics,
            'detailed_results': results
        }, f, indent=2)
    
    print(f"\nDetailed results saved to {output_file}")
    return results, avg_metrics

if __name__ == "__main__":
    print("Starting full dataset evaluation...")
    results, metrics = evaluate_full_dataset()

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
Starting full dataset evaluation...
Loading model and tokenizer from HuggingFace...


tokenizer_config.json:   0%|          | 0.00/7.36k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/2.78M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/1.67M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/11.4M [00:00<?, ?B/s]

...

Error processing example: The size of tensor a (2048) must match the size of tensor b (2542) at non-singleton dimension 2


Evaluating:  86%|████████▌ | 986/1147 [47:30<07:12,  2.69s/it]Unsloth: Input IDs of length 2276 > the model's max sequence length of 2048.
We shall truncate it ourselves. It's imperative if you correct this issue first.


Error processing example: The size of tensor a (2048) must match the size of tensor b (2276) at non-singleton dimension 2


Evaluating: 100%|██████████| 1147/1147 [55:48<00:00,  2.92s/it]


Overall Results:
Total samples processed: 1129
Successful evaluations: 1129
Failed evaluations: 18
Average ROUGE-1 F1: 0.743
Average ROUGE-2 F1: 0.673
Average ROUGE-L F1: 0.743
Average Cosine Similarity: 0.828
Result Accuracy: 0.377

Detailed results saved to evaluation_results.json
